<a href="https://colab.research.google.com/github/Car27VeP/AED_pozos_petroliferos/blob/master/Train_chatbot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycricbuzz
!pip install billboard.py
!pip install COVID19Py
!pip install pygame

In [2]:
import keras
import nltk
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
from pycricbuzz import Cricbuzz
import billboard
import time
from pygame import mixer
import COVID19Py

from nltk.stem.porter import PorterStemmer

from nltk.stem import WordNetLemmatizer

from torch.utils.data import Dataset, DataLoader

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [94]:
import time
from datetime import datetime
import pytz

# Define tu zona horaria, por ejemplo 'America/Mexico_City'
timezone = pytz.timezone('America/Mexico_City')

# Obtener la hora local
local_time = datetime.now(timezone)

In [3]:
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# NLP functions.

In [4]:
stemmer = PorterStemmer()


def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bag   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

# Dataloader setup.

In [5]:
with open('intents.json', 'r') as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        all_words.extend(w) #extend is used when you want to append a list on top of an empty list
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag) #add index to class label
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters
num_epochs = 1000
batch_size = 15
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

119 patterns
31 tags: ['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'noanswer', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']
135 unique stemmed words: ["'m", "'s", ',', '10', '19', 'a', 'about', 'age', 'am', 'anyon', 'are', 'ask', 'awesom', 'bad', 'bbye', 'be', 'best', 'bye', 'can', 'clear', 'cloudi', 'cold', 'condit', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'design', 'develop', 'do', 'dumb', 'fine', 'for', 'forecast', 'funni', 'get', 'give', 'good', 'goodby', 'googl', 'great', 'haha', 'he', 'hello', 'help', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inform', 'inspir', 'internet', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'like', 'lmao', 'lol', 'lost', 'made', 'make', 'malik', 'match', 'me'

# Multilayer Neural Network Model.

In [6]:
!pip install tensorflow
import tensorflow as tf

In [7]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(tags))
y_train.shape

(119, 31)

In [8]:
#Model
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tags),activation='softmax'))

adam=keras.optimizers.Adam(0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)
weights=model.fit(X_train,y_train,epochs=200,batch_size=10,verbose=1)
model.save('mymodel.h5',weights)

from keras.models import load_model
model = load_model('mymodel.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0224 - loss: 3.4543
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0512 - loss: 3.4068 
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0568 - loss: 3.3967
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0505 - loss: 3.3905
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1230 - loss: 3.3428
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0397 - loss: 3.3132
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0790 - loss: 3.2337
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1002 - loss: 3.1992
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1035 - loss: 3.1495
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1281 - loss: 3.0688
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1722 - loss: 3.0753
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accur

# Weather forecast

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_forecast(URL="https://weather.com/en-US/tiempo/hoy/l/bac9f66fc178da833b800af920d3f98021b442801d97761757c44e86689546b8"):


    try:
        #print(f"Solicitud de conexión a html de la página web {URL} existosa")
        req = requests.get(URL)
        soup = BeautifulSoup(req.text, 'lxml')


        place = soup.find('h1', class_='CurrentConditions--location--yub4l').text.split(',')[0]
        time = soup.find('span',class_='CurrentConditions--timestamp--LqnOd').text
        weather_condition = soup.find('div',class_="CurrentConditions--phraseValue---VS-k").text

        temp = []
        i = 0
        for i,span in enumerate(soup.find_all('span',attrs={'data-testid': 'TemperatureValue','dir': 'ltr'})):
            if i == 3:
                break
            else:
                temp.append((int(span.text[:-1]) - 32) / 1.8)
                i += 1

        current_temperature = temp[0]
        day_temperature = temp[1]
        night_temperature = temp[2]


        forecast_dict = {'place':place,
                        'time':time,
                        'weather_condition':weather_condition,
                        'current_temperature_celcius':current_temperature,
                        'day_temperature_celcius':day_temperature,
                        'night_temperature__celcius':night_temperature}

        data = pd.DataFrame([forecast_dict])
    except:
        print("No fue posible la conexión.")

    return data

# WeathE.py

In [101]:
def get_response(msg):
  sentence = tokenize(msg) #this is for intents
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])

  prediction = model.predict(X,verbose=0)[0]
  predicted_tag = tags[np.argmax(prediction)]

  if predicted_tag == 'datetime':
    timezone = pytz.timezone('America/Mexico_City')
    time = datetime.now(timezone)
    return time.strftime("%A") + ", " + time.strftime("%d %B %Y") + ", " + time.strftime("%H:%M:%S")
  else:
    for intent in intents['intents']:
      if predicted_tag == intent['tag']:
        anw = random.choice(intent['responses'])

    if predicted_tag == 'weather':
      data = get_forecast()
      complete_anw = f"{anw}{''.join(data['place'].values)}.\nThe current temperature is {data['current_temperature_celcius'].values[0]:.2f}°C.\nThe day temperature is {data['day_temperature_celcius'].values[0]:.2f}°C.\nThe night temperature is {data['night_temperature__celcius'].values[0]:.2f}°C.\nThe weather condition is {data['weather_condition'].values[0]}."
      return complete_anw
    else:

      return anw

# Production WeathE 🤖💬

In [103]:
print("Let's chat! (type 'quit' to exit)")
while True:
  sentence = input("You: ")
  if sentence == "quit":
    break

  resp = get_response(sentence)
  print(f"WeathE: {resp}")

Let's chat! (type 'quit' to exit)
You: Hi
WeathE: Hello
You: What time is it
WeathE: Tuesday, 26 November 2024, 15:14:33
You: what day is today
WeathE: Tuesday, 26 November 2024, 15:45:54
You: by
WeathE: Hello
You: bye
WeathE: See you!
You: quit


In [107]:
!git config --global user.email "carlos.pazaran@outlook.com"
!git config --global user.name "Car27VeP"

In [108]:
!git init

Reinitialized existing Git repository in /content/.git/


In [109]:
!git add .

In [110]:
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/Car27VeP/WeathE.git
!git push -u origin main

fatal: pathspec 'README.md' did not match any files
[main (root-commit) 4f77092] first commit
 25 files changed, 51346 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2024.11.25/19.12.20.690274.log
 create mode 100644 .config/logs/2024.11.25/19.12.42.183555.log
 create mode 100644 .config/logs/2024.11.25/19.12.55.136430.log
 create mode 100644 .config/logs/2024.11.25/19.12.56.407212.log
 create mode 100644 .config/logs/2024.11.25/19.13.09.316504.log
 create mode 100644 .config/logs/2024.11.25/19.13.10.119162.log
 create mod